# demo for KPI ENGINE

This notebook is intentended to showcase the capabilities of the kpi engine for milestone 2.

Note:the docker container must be up and running

## enhanced calculation capabilities

New feature: unit calculation support.

In [11]:
import requests
from pprint import pprint

url='http://localhost:8000/calculate/'
request_data={
    'machine_id':'ast-xpimckaf3dlf',
    'start_date':'2024-10-14',
    'end_date':'2024-10-19',
    'expression':'avg(good_cycles)',
}
request_result=requests.get(url=url,params=request_data).json()

print(f"measuring unit :{request_result['unit']}")
pprint(request_result,indent=4)


measuring unit :#
{   'code': 0,
    'end_time': '2024-10-19 00:00:00',
    'expression': 'avg(good_cycles)',
    'reason': '',
    'result': {'2024-10-14/2024-10-19': {'values': 10077.2}},
    'start_time': '2024-10-14 00:00:00',
    'unit': '#'}


Now time aggregations are possible.
- on a weekly basis
- on a monthly basis

Also the output is much more structured

In [12]:
request_data['expression']="sum(good_cycles)"
request_data['start_date']="2024-09-01"
request_data['end_date']="2024-10-30"
request_data['unit']='w'

request_result=requests.get(url=url,params=request_data).json()

pprint(request_result['result'],indent=4)

{   '2024-09-01/2024-09-01': {'values': 0},
    '2024-09-02/2024-09-08': {'values': 35692.0},
    '2024-09-09/2024-09-15': {'values': 38216.0},
    '2024-09-16/2024-09-22': {'values': 54193.0},
    '2024-09-23/2024-09-29': {'values': 32591.0},
    '2024-09-30/2024-10-06': {'values': 37535.0},
    '2024-10-07/2024-10-13': {'values': 25077.0},
    '2024-10-14/2024-10-20': {'values': 51111.0},
    '2024-10-21/2024-10-27': {'values': 0},
    '2024-10-28/2024-10-30': {'values': 0}}


In [13]:
request_data['expression']="sum(good_cycles)"
request_data['start_date']="2024-09-01"
request_data['end_date']="2024-10-30"
request_data['unit']='m'

request_result=requests.get(url=url,params=request_data).json()

pprint(request_result['result'])

{'2024-09-01/2024-09-30': {'values': 160692.0},
 '2024-10-01/2024-10-30': {'values': 113723.0}}


Now it is possible to use values from the knowledge base.

In [24]:
request_data['expression']="sum(machine_down_time_percentage)"
request_data['start_date']="2024-10-14"
request_data['end_date']="2024-10-30"
request_data['unit']='w'

request_result=requests.get(url=url,params=request_data).json()

pprint(request_result['result'])

{'2024-10-14/2024-10-20': {'values': 150.1702},
 '2024-10-21/2024-10-27': {'values': 0},
 '2024-10-28/2024-10-30': {'values': 0}}


Both base KPIs and KPIs present in the knowledge base can be used together.

In [15]:
request_data['expression']="machine_down_time_percentage+good_cycles"
request_data['start_date']="2024-10-14"
request_data['end_date']="2024-10-19"
request_data['unit']='d'

request_result=requests.get(url=url,params=request_data).json()

pprint(request_result['result'])

{'2024-10-14/2024-10-19': {'time': ['2024-10-14T00:00:00Z',
                                    '2024-10-18T00:00:00Z',
                                    '2024-10-17T00:00:00Z',
                                    '2024-10-15T00:00:00Z',
                                    '2024-10-16T00:00:00Z'],
                           'values': [6230.0233,
                                      4641.4652,
                                      13530.5848,
                                      16495.5894,
                                      9594.5509]}}


# AI agent integration (mocked)

The AI agent will use the same interface as the others here are some examples of possible interactions:
1. specify a KPI for a single machine in a day

In [16]:
request_data['expression']="machine_down_time_percentage"
request_data['start_date']="2024-10-14"
request_data['end_date']="2024-10-15"
request_data['unit']='d'

request_result=requests.get(url=url,params=request_data).json()

pprint(request_result['result'])

{'2024-10-14/2024-10-15': {'time': ['2024-10-14T00:00:00Z'],
                           'values': [13.0233]}}


2. specify a single KPI for a machine in a time range

In [17]:
request_data['expression']="machine_down_time_percentage"
request_data['start_date']="2024-10-14"
request_data['end_date']="2024-10-19"
request_data['unit']='d'

request_result=requests.get(url=url,params=request_data).json()

pprint(request_result['result'])

{'2024-10-14/2024-10-19': {'time': ['2024-10-14T00:00:00Z',
                                    '2024-10-18T00:00:00Z',
                                    '2024-10-17T00:00:00Z',
                                    '2024-10-15T00:00:00Z',
                                    '2024-10-16T00:00:00Z'],
                           'values': [13.0233,
                                      28.4652,
                                      17.5848,
                                      9.5894,
                                      37.5509]}}


3. specify a single aggregation function on KPI for a single machine

In [18]:
request_data['expression']="machine_down_time_percentage"
request_data['start_date']="2024-10-14"
request_data['end_date']="2024-10-19"
request_data['operation']='sum'
request_result=requests.get(url=url,params=request_data).json()

pprint(request_result['result'])

{'2024-10-14/2024-10-19': {'time': ['2024-10-14T00:00:00Z',
                                    '2024-10-18T00:00:00Z',
                                    '2024-10-17T00:00:00Z',
                                    '2024-10-15T00:00:00Z',
                                    '2024-10-16T00:00:00Z'],
                           'values': [13.0233,
                                      28.4652,
                                      17.5848,
                                      9.5894,
                                      37.5509]}}


4. specify a time-aggregation for  a KPI on a single machine

In [19]:
request_data['expression']="machine_down_time_percentage"
request_data['start_date']="2024-10-10"
request_data['end_date']="2024-10-20"
request_data['unit']='w'
request_data['operation']='sum'
request_result=requests.get(url=url,params=request_data).json()

pprint(request_result['result'])

{'2024-10-10/2024-10-13': {'time': ['2024-10-11T00:00:00Z',
                                    '2024-10-12T00:00:00Z',
                                    '2024-10-10T00:00:00Z'],
                           'values': [26.3336, 28.3687, 11.6377]},
 '2024-10-14/2024-10-20': {'time': ['2024-10-14T00:00:00Z',
                                    '2024-10-18T00:00:00Z',
                                    '2024-10-19T00:00:00Z',
                                    '2024-10-17T00:00:00Z',
                                    '2024-10-15T00:00:00Z',
                                    '2024-10-16T00:00:00Z'],
                           'values': [13.0233,
                                      28.4652,
                                      17.5848,
                                      9.5894,
                                      37.5509,
                                      43.9566]}}


# alert generation & monitoring capabilities

The kpi engine now features alert generation & monitoring capabilities.

In [20]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from app.Alert_Monitor.alert_monitor import test_alerts
test_alerts()

evaluating allert with expression avg(good_cycles)>10000
date 2024-09-30 00:00:00 no alert detected
evaluating allert with expression avg(good_cycles)>10000
date 2024-10-01 00:00:00 no alert detected
evaluating allert with expression avg(good_cycles)>10000
date 2024-10-02 00:00:00 alert detected
alert fired
evaluating allert with expression avg(good_cycles)>10000
date 2024-10-03 00:00:00 no alert detected
evaluating allert with expression avg(good_cycles)>10000
date 2024-10-04 00:00:00 no alert detected
evaluating allert with expression avg(good_cycles)>10000
date 2024-10-05 00:00:00 no alert detected
evaluating allert with expression avg(good_cycles)>10000
date 2024-10-06 00:00:00 no alert detected
evaluating allert with expression avg(good_cycles)>10000
date 2024-10-07 00:00:00 no alert detected
evaluating allert with expression avg(good_cycles)>10000
date 2024-10-08 00:00:00 no alert detected
evaluating allert with expression avg(good_cycles)>10000
date 2024-10-09 00:00:00 no alert 